# Setting up environment

In [1]:
!pip install -U langchain langchain-community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


# Loading the models

In [2]:
#Only needed when using Google Colab

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir(r"/content/drive/My Drive/Colab Notebooks/Predictive Analytics/Francisco/Combined model")
!ls

Mounted at /content/drive
 assets.pkl	        emotions_model.keras   __pycache__
 Combined_model.ipynb   emotion_utils.py       sentiment_model.keras
 custom_funtions.py     labels_2.csv	       tokenizer.pkl
'Demo Diary.ipynb'      mood_diary.csv	       Try_using_function.ipynb


In [3]:
import pickle
from keras.datasets import imdb
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from emotion_utils import predict_sent_emotions
from custom_funtions import LearnablePositionalEncoding, focal_loss_with_penalty, f1_metric


model = load_model("emotions_model.keras", custom_objects={
    "LearnablePositionalEncoding": LearnablePositionalEncoding,
    "focal_loss_with_penalty": focal_loss_with_penalty,
    "f1_metric": f1_metric
})

# Load models
model_emotion = load_model("emotions_model.keras", custom_objects={
    "LearnablePositionalEncoding": LearnablePositionalEncoding,
    "focal_loss_with_penalty": focal_loss_with_penalty,
    "f1_metric": f1_metric
})

model_sentiment = load_model("sentiment_model.keras")

# Load tokenizer and other assets
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("assets.pkl", "rb") as f:
    data = pickle.load(f)
    word_index = data['word_index']
    emotion_columns = data['emotion_columns']

# Predict
text = "I cried during the whole movie of sadness, but it was really good"
result = predict_sent_emotions(text, model_emotion, model_sentiment, tokenizer,
                               word_index, emotion_columns, max_len=100, threshold=0.5)

print(result)
# Output: ['Positive', ['Joy', 'Excitement']]


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 24 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 959ms/step
[['Positive', ['sadness']]]


# Connect to Claude API

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [1]:
# Install Anthropics SDK
!pip install anthropic

import anthropic
import os

# Set API key
#client = anthropic.Anthropic(
    #api_key= "IN THE REPORT"
)

SyntaxError: unmatched ')' (391973923.py, line 10)

## Getting the data from the audio files

In [16]:
import pandas as pd

# Load data from CSV
input_data = pd.read_csv('labels_2.csv')

# Extract only the first column
input_data_text = input_data.iloc[:, 3]
input_data_sentiment = input_data.iloc[:, 1]
input_data_emotion = input_data.iloc[:, 2]

# Convert the first column to a list
input_data_text = input_data_text.tolist()


# Comparing own model with claude

In [30]:
# List of test sentences
example_texts = [
    "The movie was fantastic! I loved every second.",
    "What a terrible film. I walked out halfway.",
    "It was okay, not great but not bad either.",
    "I cried through the whole thing, it was so emotional.",
    "I'm still laughing! That was hilarious."
]

# Prompt template for binary sentiment classification
def binary_classification_prompt(text):
    return f"""Classify the sentiment of the following sentence as Positive or Negative. Just answer with one word: Positive or Negative.\n\n{text}"""

# Prompt template for multi-class emotion detection
def multi_class_emotion_prompt(text):
    return f"""Classify the sentiment of the following sentence as the suitable emotions out of the following list: admiration, amusement, anger, annoyance,
               approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy,
                love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral.
                Just answer with the list of all detected emotions and nothing else.\n\n{text}"""


#Our own model
print("\n=== Sentiment Analysis with our own model ===\n")
predictions = predict_sent_emotions(input_data_text, model_emotion, model_sentiment, tokenizer,
                               word_index, emotion_columns, max_len=100, threshold=0.5)

#accuracy of sentiment
sentiments_pred = [prediction[0] for prediction in predictions]
sentiments_true = input_data_sentiment.tolist()
correct_sentiments = sum(
    1 for pred, true in zip(sentiments_pred, sentiments_true) if pred.lower() == true.lower()
)

# Calculate accuracy
accuracy_sentiment = correct_sentiments / len(sentiments_true)

#accuracy for emotion
emotions_pred = [prediction[1] for prediction in predictions]
emotions_true = input_data_emotion.tolist()
correct_emotions = sum(
    1 for pred_list, true in zip(emotions_pred, emotions_true)
    if any(p.lower() == true.lower() for p in pred_list)
)

# Calculate accuracy
accuracy_emotions = correct_emotions / len(emotions_true)

print(f"Emotion Accuracy: {accuracy_emotions:.2%}")
print(f"Sentiment Accuracy: {accuracy_sentiment:.2%}")

#get the emotions that are predicted as 1
#predicted_emotions = predictions.loc[:, (predictions != 0).any(axis=0)]



=== Sentiment Analysis with our own model ===

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Emotion Accuracy: 13.17%
Sentiment Accuracy: 65.37%


In [46]:
import string
correct_emotions = 0
correct_sentiments = 0

print("\n=== Sentiment Analysis with Claude ===\n")
# Run Claude over the list
for i, text in enumerate(input_data_text):
    # First call: Binary classification
    binary_response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=20,
        messages=[{"role": "user", "content": binary_classification_prompt(text)}]
    )
    binary_prediction = binary_response.content[0].text.strip()
    binary_prediction = binary_prediction.strip().lower().rstrip(string.punctuation)
    if binary_prediction == input_data_sentiment[i].lower():
      correct_sentiments += 1

    # Second call: Emotion detection
    emotion_response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=100,
        messages=[{"role": "user", "content": multi_class_emotion_prompt(text)}]
    )
    emotion_prediction = emotion_response.content[0].text.strip()

    predicted_emotions = [
        e.strip().lower().rstrip(string.punctuation)
        for e in emotion_prediction.split(',')
    ]

    # Normalize true emotion
    true_emotion = input_data_emotion[i].strip().lower().rstrip(string.punctuation)

    if true_emotion in predicted_emotions:
        correct_emotions += 1

# Calculate accuracy
accuracy_sentiment = correct_sentiments / len(sentiments_true)
accuracy_emotions = correct_emotions / len(emotions_true)

print(f"Emotion Accuracy: {accuracy_emotions:.2%}")
print(f"Sentiment Accuracy: {accuracy_sentiment:.2%}")


=== Sentiment Analysis with Claude ===

Emotion Accuracy: 86.34%
Sentiment Accuracy: 96.59%
